<a href="https://colab.research.google.com/github/stefphd/bicycle-lnn/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>